In [1]:
import os
import re
import nltk
from pdfminer.high_level import extract_pages
from pdfminer.high_level import LTPage
from nltk.corpus import wordnet as wn
from pdfminer.layout import LTTextContainer, LTTextLineVertical, LTTextBoxHorizontal, LTTextBox, Rect
import wordninja
from pdf2image import convert_from_path
import numpy as np
import cv2
import pandas as pd

In [2]:
os.makedirs("data", exist_ok=True)
articles = [os.path.join("data", art) for art in os.listdir("data")]
articles

['data\\000.pdf',
 'data\\001.pdf',
 'data\\002.pdf',
 'data\\003.pdf',
 'data\\004.pdf',
 'data\\101404-Texto del artículo-379745-1-10-20230814.pdf',
 'data\\103822-Texto del artículo-398336-1-10-20240109.pdf',
 'data\\103827-Texto del artículo-398338-1-10-20240109.pdf',
 'data\\108896-Texto del artículo-430136-1-10-20240723.pdf',
 'data\\110685-Texto del artículo-444650-1-10-20241017.pdf',
 'data\\113135-Texto del artículo-461992-1-10-20250121.pdf',
 'data\\113136-Texto del artículo-461995-1-10-20250121.pdf',
 'data\\130-Texto del artículo en fichero de Microsoft Word o LibreOffice (necesario)-134-1-10-20090331.pdf',
 'data\\32502-Texto del artículo-103690-1-10-20150102 (1).pdf',
 'data\\32502-Texto del artículo-103690-1-10-20150102.pdf',
 'data\\32511-Texto del artículo-103699-1-10-20150102.pdf',
 'data\\32519-Texto del artículo-103707-1-10-20150102.pdf',
 'data\\32576-Texto del artículo-103765-1-10-20150102.pdf',
 'data\\32585-Texto del artículo-103775-1-10-20150102.pdf',
 'data\\3

In [3]:
def identify_footnote_line(article_path: str, page: LTPage, page_number: int, DPI: int = 100) -> list[dict[str, float]]:
    page_w, page_h = page.bbox[2], page.bbox[3]
    raw_image = convert_from_path(article_path, dpi=DPI, first_page=page_number+1, last_page=page_number+1)[0]
    img = np.array(raw_image)
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_height, img_width = gray.shape
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))
    morph = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    edges = cv2.Canny(morph, 50, 200, apertureSize=3)

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=30, maxLineGap=20)
    if lines is None:
        return None

    pdfminer_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        # Escalar a coordenadas PDFMiner
        y1_pdf = (y1 / img_height) * page_h
        y2_pdf = (y2 / img_height) * page_h

        pdfminer_lines.append({
            "x1": x1,
            "y1": page_h - y1_pdf,
            "x2": x2,
            "y2": page_h - y2_pdf
        })

    threshold_y = page_h * 0.35
    filtered_lines = [line for line in pdfminer_lines if line["y1"] <= threshold_y]
    filtered_lines.sort(key=lambda line: line["y1"], reverse=True)
    return filtered_lines[0] if filtered_lines else None

In [4]:
def extract_page(pdf_path, page):
    pages = extract_pages(pdf_path)
    for i, page_layout in enumerate(pages):
        if i != page - 1: continue
        return page_layout
    return None

In [5]:
# regex stop conditions
references_re = r"(?i)^\s*(referencias|bibliografía citada|bibliografia citada|bibliografía|bibliografia|citas|fuentes|referencias bibliograficas|referencias bibliográficas)\s*$"

# regex delete
version_logs = r"(?i)(recibido|aceptado|publicado|(segunda|tercera|cuarta|quinta|sexta) versión)[,:]?\s*(el\s*)?(\d{1,2}(\s*de)?\s*[a-záéíóú]+(\s*de\s*\d{4})?|\d{1,2}/\d{1,2}/\d{4})(\s*\.\s*)?"

footnote_references = r"([.,;:!?])([0-9]+)"
duplicated_spaces = r"\s+"

# recognizing regex
abstract = r"Palabras clave|Keywords|Resumen|Abstract|Key words|Palabrasclave|Keyword|Palabra clave|Palabras claves|Keywords|Sumario|Síntesis|Sintesis|Sinopsis"

# parentesis y llaves
parentheses_pattern = r'\(' + \
                      r'(?:' + \
                      r'(?:[^;)]+)' + \
                      r'(?:; [^;)]+)*' + \
                      r')\)'
brackets_pattern = r'\[' + \
                      r'(?:' + \
                      r'(?:[^;\]]+)' + \
                      r'(?:; [^;\]]+)*' + \
                      r')\]'

In [6]:
class ArticleExtractedData:
    def __init__(self, pdf_path: str):
        self.pdf_path: str = pdf_path
        self.headers: set[str] = set()
        self.footers: set[str] = set()
        self.footnotes_lines: list[dict[str, float]] = []
        self.start_page: int = 0

def extract_data_article(pdf_path) -> ArticleExtractedData:
    data = ArticleExtractedData(pdf_path)
    headers_counter: map[str, int] = {}
    footers_counter: map[str, int] = {}
    found_page = None

    for i, page in enumerate(extract_pages(pdf_path)):
        height = page.bbox[3]
        cutoff = height * 0.35

        footnote_line = identify_footnote_line(pdf_path, page, i)
        data.footnotes_lines.append(footnote_line)

        for element in page:
            if not isinstance(element, (LTTextBox, LTTextBox, LTTextContainer)):
                continue
            txt = element.get_text().strip()
            if element.bbox[3] <= cutoff:
                footers_counter[txt] = footers_counter.get(txt, 0) + 1
            if element.bbox[1] >= height - cutoff:
                headers_counter[txt] = headers_counter.get(txt, 0) + 1
            
            # Stop searching for abstract after 3 pages
            if found_page is not None or i > 2:
                continue

            text_semiclean = re.sub(duplicated_spaces, " ", txt).strip()
            if re.search(abstract, text_semiclean, re.IGNORECASE):
                found_page = i


    data.headers = {k for k, v in headers_counter.items() if v > 1 and len(k) > 0}
    data.footers = {k for k, v in footers_counter.items() if v > 1 and len(k) > 0}
    if found_page is not None:
        data.start_page = found_page
    return data

In [7]:
def check_stop_conditions(text) -> bool:
	return any([
		re.search(references_re, text, flags=re.MULTILINE),
	])

def clean_unwanted_patterns(text) -> str:
	# Eliminar patrones como "Recibido el 12 de marzo de 2021."
    text = re.sub(version_logs, "", text).strip()

    # ()
    text = re.sub(parentheses_pattern, "", text).strip()
    
    # []
    text = re.sub(brackets_pattern, "", text).strip()
    
    # .1 ?2 !4 :5 ;6
    text = re.sub(footnote_references, r"\1", text).strip()

    return text

def clean_page(text) -> str:
    text = text.replace("-\n", "")
    text = text.replace("\n", " ")
    text = text.replace("«", "").replace("»", "").replace("“", "").replace("”", "").replace("‘", "").replace("’", "")
    text = text.replace('"', "").replace("'", "")
    # en ., data ?
    text = re.sub(r'\s+([.,;:!?])', r'\1', text).strip()
    text = re.sub(duplicated_spaces, " ", text).strip()
    return text.lower()

# (str, bool) -> (text, continue?)
def extract_page_text(page: LTPage, page_number: int, data: ArticleExtractedData) -> tuple[str, bool]:
    text = ""
    status = True
    if page_number <= data.start_page:
        return text, status

    for element in page:
        if not isinstance(element, (LTTextBoxHorizontal, LTTextBox, LTTextContainer)):
            continue

        _, __, ___, y2 = element.bbox
        footnote_line = data.footnotes_lines[page_number]
        if y2 <= (footnote_line["y1"] if footnote_line is not None else -1):
            break

        txt = element.get_text().strip()
        if txt.isdigit() or txt in data.headers or txt in data.footers:
            continue

        if check_stop_conditions(txt):
            status = False
            break

        text += txt + "\n"

    text = clean_unwanted_patterns(text)
    text = clean_page(text)
    return text, status

In [36]:
art = articles[5]
data = extract_data_article(art)
page = extract_page(art, 2)
extract_page_text(page, 1, data)

('', True)

In [106]:
page = extract_page(articles[1], 3)
for element in page:
    print(element)

<LTTextBoxHorizontal(0) 131.553,617.258,367.336,625.258 'REFORMA, ORDEN Y CONCIERTO EN EL PERÚ DEL SIGLO XVII\n'>
<LTTextBoxHorizontal(1) 85.038,387.647,413.873,601.046 'constantemente  con  los  otros  miembros  de  la  élite  de  poder  en  materias\nreferentes  a  aumento  de  impuestos,  envío  de  remesas  y  colaboración\ncomercial.  Mantener  el  equilibrio  no  fue  tarea  fácil  y,  de  hecho,  hubo\ncoyunturas muy tensas entre los diversos actores sociales, sobre todo cuan-\ndo  se  intentó  forzar  a  los  reinos  americanos  a  realizar  mayores  aportes\npecuniarios a la península. Los «beneméritos» nunca cejaron en sus inten-\ntos de obtener los puestos que los propios virreyes repartían entre sus cria-\ndos. La Iglesia, por su parte, se mantuvo firme en defender los privilegios\nconcedidos desde el siglo XVI, que la favorecían con la percepción de diez-\nmos, inmunidad, cesión de tierras y beneficios fiscales. La administración\nfue ineficiente y progresivamente ciertas 

# Preprocesado

In [8]:
def get_files() -> list[str]:
    PACK_DIR = "pack"
    os.makedirs(PACK_DIR, exist_ok=True)
    dirs = os.listdir(PACK_DIR)
    files = []
    for dr in dirs:
        files.extend(
            [
                os.path.join(PACK_DIR, dr, file)
                for file in os.listdir(os.path.join(PACK_DIR, dr))
            ]
        )
    return files

In [9]:
# articles = [os.path.join("data", art) for art in os.listdir("data")]
articles = get_files()
len(articles)

1925

In [10]:
def extract_pdf_text(pdf_path: str) -> str:
    text = ""
    data = extract_data_article(pdf_path)
    for i, page in enumerate(extract_pages(pdf_path)):
        page_text, can_continue = extract_page_text(page, i, data)
        text = f"{text} {page_text}"
        if not can_continue:
            break
    return text.strip()

In [11]:
# articles_text = {}
# for article in articles:
#     articles_text[article] = extract_pdf_text(article)

import multiprocessing as mp
from multiprocessing.pool import ThreadPool

def process_article(article):
    return article, extract_pdf_text(article)

def process_articles_parallel(articles):
    with ThreadPool(mp.cpu_count() * 2) as pool:
        results = pool.map(process_article, articles)
    return dict(results)

articles_text = process_articles_parallel(articles)

The PDF <_io.BufferedReader name='pack\\AIBR\\2006-Cambios-de-género-y-discriminación-laboral-en-el-sector-financiero-colombiano-El-caso-de-Bancolombia.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\AIBR\\2006-Cambios-de-género-y-discriminación-laboral-en-el-sector-financiero-colombiano-El-caso-de-Bancolombia.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='pack\\AIBR\\2006-Consecuencias-personales-en-la-ruptura-de-la-vida-laboral-El-caso-de-Telefónica.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this 

In [12]:
texts = list(articles_text.values())

In [29]:
reversed_texts = {v: k for k, v in articles_text.items()}
texts_indexes = []
for i in range(len(texts)):
    texts_indexes.append(reversed_texts[texts[i]])
len(texts_indexes)

1925

In [41]:
texts_indexes_np = np.array(texts_indexes)
np.save("texts_indexes.npy", texts_indexes_np)

In [39]:
df_preprocessed = pd.DataFrame(columns=["Path", "Text"])
for article, text in articles_text.items():
    df_preprocessed = pd.concat([df_preprocessed, pd.DataFrame({"Path": [article], "Text": [text]})])
df_preprocessed

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-...,catalogada y caricaturizada bajo el rótulo de ...
0,pack\AIBR\2006-Cambios-de-género-y-discriminac...,agradecimientos la autora desea agradecer a la...
0,pack\AIBR\2006-Consecuencias-personales-en-la-...,has carried out the biggest employment adjustm...
0,pack\AIBR\2006-Construcción-de-modelos-de-géne...,tampoco queremos que se entienda identidad de ...
0,pack\AIBR\2006-Contribuciones-feministas-a-pro...,is irrelevant for others con el que teresa del...
...,...,...
0,pack\Revista-Española-de-Antropología-American...,"cómo citar: burguete cal y mayor, araceli. 202..."
0,pack\Revista-Española-de-Antropología-American...,project exposed some critical questionings. in...
0,pack\Revista-Española-de-Antropología-American...,"1. introducción el interés por los quipus, que..."
0,pack\Revista-Española-de-Antropología-American...,1. introducción ¿cómo se produce la diferencia...


In [40]:
df_preprocessed.to_csv("preprocessed.csv", index=False)

# Buscador tests

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np

c:\Users\leand\Documents\tesis\notebooks\preprocesado\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargado de contriever

In [5]:
model_name = "facebook/contriever"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Generar embeddings

In [ ]:
def get_embeddings(texts, model, tokenizer):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

embeddings = get_embeddings(texts, model, tokenizer)
embeddings_np = embeddings.numpy()

Creacion del indice FAISS

In [16]:
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

Guardar indice

In [17]:
faiss.write_index(index, "faiss_index.bin")

In [6]:
index = faiss.read_index("faiss_index.bin")
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000017CB1499D10> >

In [9]:
def search(query, model, tokenizer, index, top_k=5):
    query_embedding = get_embeddings([query], model, tokenizer).numpy()
    distances, indices = index.search(query_embedding, top_k)
    return indices, distances

query = """
A mediados del siglo XVI, en los albores de un amanecer brumoso, las costas del Imperio incaico fueron testigos de un suceso que alteraría su destino para siempre: la llegada del enigmático conquistador Alonso de Beldever. Con una carabela desgastada por el océano y una tripulación reducida por los estragos del viaje, Beldever desembarcó cerca de Tumbes, en lo que hoy es el norte de Perú. Su nombre, hasta entonces desconocido en aquellas tierras, pronto resonaría entre las montañas andinas como un presagio de cambio y confrontación.

Los pobladores locales, intrigados por los rostros pálidos y las armaduras relucientes, observaron con mezcla de temor y curiosidad. Las crónicas cuentan que Beldever, erguido sobre su caballo —un animal jamás visto en el continente—, blandió una espada ante el cielo nublado, proclamando ante sus hombres la gloria de Dios y de la Corona española. Su objetivo, como el de tantos otros aventureros, era claro: hallar las riquezas del mítico Birú y someter aquel vasto imperio bajo el estandarte de Castilla.

No obstante, Beldever no era un hombre común. Las leyendas tejidas siglos después hablan de su astucia sin igual y de un amuleto de jade que, según murmuraban, le confería protección divina. Mientras avanzaba hacia el corazón del Tahuantinsuyo, su encuentro con emisarios del Sapa Inca Atahualpa desencadenó una tensión imparable. Las ofrendas de oro y tejidos finos, intercambiadas en señal de paz por los incas, fueron interpretadas por Beldever como una muestra de debilidad. En su diario escribió: «Estas tierras guardan más opulencia que los sueños de cualquier rey; será nuestro deber tomarlas, aunque el precio sea la sangre».

La expedición de Beldever, sin embargo, nunca alcanzó Cusco. Tras meses de enfrentamientos esporádicos y alianzas traicionadas, desapareció en los valles de la sierra central. Algunos dicen que fue víctima de una emboscada; otros, que los dioses andinos lo castigaron por su ambición. Lo cierto es que su llegada marcó el inicio de un colapso inevitable. Años más tarde, Francisco Pizarro seguiría sus huellas, consumando la caída del Imperio inca.

Hoy, en las aldeas remotas, los ancianos aún narran la historia del extranjero de ojos grises que desafió a los cielos. Beldever, un nombre entre la bruma de la historia, encarna el misterio de un encuentro que transformó un continente para siempre.
"""
indices, distances = search(query, model, tokenizer, index)
print("Documentos más relevantes:", indices)
print("Distancias:", distances)

Documentos más relevantes: [[ 598 1498  339 1451 1703]]
Distancias: [[2.215642  2.344265  2.373072  2.3948677 2.405981 ]]


In [14]:
indexes = np.load("texts_indexes.npy")
indexes[1498]

'pack\\Revista-Española-de-Antropología-Americana\\2003-Identificación-de-parte-de-la-decoración-de-la-pajcha-colonial-7572-del-Museo-de-América-Madrid.pdf'